# Data Aggregation and Group Operations

In [1]:
import numpy as np
from numpy.random import randn

import pandas as pd
from pandas import Series, DataFrame

## GroupBy Mechanics

In [2]:
df = DataFrame({
    'key1': ['a','a','b','b','a'],
    'key2': ['A','B','A','B','A'],
    'data1': np.arange(0,5),
    'data2': np.arange(0,10,2)
})
df

,data1,data2,key1,key2
0,0,0,a,A
1,1,2,a,B
2,2,4,b,A
3,3,6,b,B
4,4,8,a,A


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     A       2
      B       1
b     A       2
      B       3
Name: data1, dtype: int64

In [6]:
means.unstack()

key2,A,B
key1,,
a,2,1
b,2,3


In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.666667,3.333333
b,2.500000,5.000000


In [8]:
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    A         2      4
     B         1      2
b    A         2      4
     B         3      6

### Iterating Over Groups

In [9]:
for name, group in df.groupby('key1'):
    print name
    print group

a
   data1  data2 key1 key2
0      0      0    a    A
1      1      2    a    B
4      4      8    a    A
b
   data1  data2 key1 key2
2      2      4    b    A
3      3      6    b    B


In [10]:
for type, group in df.groupby(df.dtypes, axis=1):
    print type
    print group

int64
   data1  data2
0      0      0
1      1      2
2      2      4
3      3      6
4      4      8
object
  key1 key2
0    a    A
1    a    B
2    b    A
3    b    B
4    a    A


### Selecting a Column or Subset of Columns

In [11]:
df.groupby('key1')['data1'].count()

key1
a    3
b    2
Name: data1, dtype: int64

In [12]:
df['data1'].groupby(df['key1']).count()

key1
a    3
b    2
Name: data1, dtype: int64

### Grouping with Dicts and Series

In [13]:
people = DataFrame(
    np.arange(25).reshape((5,5)),
    columns=['a','b','c','d','e'],
    index=['one','two','three','four','five']
)
people

,a,b,c,d,e
one,0,1,2,3,4
two,5,6,7,8,9
three,10,11,12,13,14
four,15,16,17,18,19
five,20,21,22,23,24


In [14]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

people.groupby(mapping, axis=1).sum()

,blue,red
one,5,5
two,15,20
three,25,35
four,35,50
five,45,65


In [15]:
map_series = Series(mapping)
people.groupby(map_series, axis=1).sum()

,blue,red
one,5,5
two,15,20
three,25,35
four,35,50
five,45,65


### Grouping with Functions

In [16]:
people.groupby(len).sum()

,a,b,c,d,e
3,5,7,9,11,13
4,35,37,39,41,43
5,10,11,12,13,14


In [17]:
people.groupby(lambda x: x[0]).sum()

,a,b,c,d,e
f,35,37,39,41,43
o,0,1,2,3,4
t,15,17,19,21,23


### Grouping by Index Levels

In [18]:
columns = pd.MultiIndex.from_arrays(
    [['US','US','US','JP','JP'],
    [1,3,5,1,3]], names=['city', 'tenor'])

hier_df = DataFrame(np.arange(25).reshape((5,5)),
                   columns=columns)
hier_df

city   US          JP    
tenor   1   3   5   1   3
0       0   1   2   3   4
1       5   6   7   8   9
2      10  11  12  13  14
3      15  16  17  18  19
4      20  21  22  23  24

In [19]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
4,2,3


## Data Aggregation

In [20]:
grouped = df.groupby('key1')
for key, group in grouped:
    print key
    print group

a
   data1  data2 key1 key2
0      0      0    a    A
1      1      2    a    B
4      4      8    a    A
b
   data1  data2 key1 key2
2      2      4    b    A
3      3      6    b    B


In [21]:
grouped['data1'].quantile(0.5)

key1
a    1.0
b    2.5
Name: data1, dtype: float64

In [22]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4,8
b,1,2


In [23]:
grouped['data1'].mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [24]:
grouped['data1'].median()

key1
a    1.0
b    2.5
Name: data1, dtype: float64

### Column-wise and Multiple Function Application

In [25]:
tips = pd.read_csv('ch08/tips.csv')
tips[:3]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [26]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:3]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587


In [27]:
grouped = tips.groupby(['sex','smoker'])

In [28]:
grouped['tip_pct'].agg('mean').unstack()

smoker,No,Yes
sex,,
Female,0.156921,0.182150
Male,0.160669,0.152771


In [29]:
grouped['tip_pct'].agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [30]:
grouped['tip_pct'].agg([
    ('avarage', 'mean'), ('standard deviation','std')
])

avarage  standard deviation
sex    smoker                              
Female No      0.156921            0.036421
       Yes     0.182150            0.071595
Male   No      0.160669            0.041849
       Yes     0.152771            0.090588

In [31]:
grouped['tip_pct', 'total_bill'].agg(['mean', 'std'])

tip_pct           total_bill          
                   mean       std       mean       std
sex    smoker                                         
Female No      0.156921  0.036421  18.105185  7.286455
       Yes     0.182150  0.071595  17.977879  9.189751
Male   No      0.160669  0.041849  19.791237  8.726566
       Yes     0.152771  0.090588  22.284500  9.911845

In [32]:
grouped.agg({'tip': 'max', 'size': 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

### Returning Aggregated Data in “unindexed” Form

In [33]:
tips.groupby(['sex','smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


## Group-wise Operations and Transformations

In [34]:
df

,data1,data2,key1,key2
0,0,0,a,A
1,1,2,a,B
2,2,4,b,A
3,3,6,b,B
4,4,8,a,A


In [35]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,1.666667,3.333333
b,2.500000,5.000000


In [36]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,0,0,a,A,1.666667,3.333333
1,1,2,a,B,1.666667,3.333333
4,4,8,a,A,1.666667,3.333333
2,2,4,b,A,2.500000,5.000000
3,3,6,b,B,2.500000,5.000000


In [37]:
people.groupby(len).mean()

,a,b,c,d,e
3,2.5,3.5,4.5,5.5,6.5
4,17.5,18.5,19.5,20.5,21.5
5,10.0,11.0,12.0,13.0,14.0


In [39]:
people.groupby(len).transform('mean')

,a,b,c,d,e
one,2.5,3.5,4.5,5.5,6.5
two,2.5,3.5,4.5,5.5,6.5
three,10.0,11.0,12.0,13.0,14.0
four,17.5,18.5,19.5,20.5,21.5
five,17.5,18.5,19.5,20.5,21.5


In [41]:
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(len).transform(demean)
demeaned

,a,b,c,d,e
one,-2.5,-2.5,-2.5,-2.5,-2.5
two,2.5,2.5,2.5,2.5,2.5
three,0.0,0.0,0.0,0.0,0.0
four,-2.5,-2.5,-2.5,-2.5,-2.5
five,2.5,2.5,2.5,2.5,2.5


### Apply: General split-apply-combine

In [60]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [61]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [62]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [65]:
frame = DataFrame({'data1': randn(1000), 'data2': randn(1000)})

In [67]:
factor = pd.cut(frame.data1, 4)
factor[:4]

0     (-0.1, 1.534]
1    (-1.735, -0.1]
2     (-0.1, 1.534]
3    (-1.735, -0.1]
Name: data1, dtype: category
Categories (4, object): [(-3.376, -1.735] < (-1.735, -0.1] < (-0.1, 1.534] < (1.534, 3.169]]

In [70]:
grouped = frame['data2'].groupby(factor)
grouped.apply(lambda x: x.describe()).unstack()

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
"(-3.376, -1.735]",33.0,-0.130192,0.935356,-2.178644,-0.860475,-0.057895,0.439782,1.891904
"(-1.735, -0.1]",409.0,-0.012401,0.986294,-2.426193,-0.781090,-0.028792,0.630485,2.724256
"(-0.1, 1.534]",496.0,-0.000415,1.033471,-2.755748,-0.648933,0.003076,0.691363,2.940230
"(1.534, 3.169]",62.0,-0.112845,1.015825,-2.160513,-0.722275,-0.268551,0.253356,2.388897


In [73]:
grouping = pd.qcut(frame.data1, 4, labels=False)
grouping[:4]

0    3
1    1
2    3
3    1
Name: data1, dtype: int64

In [74]:
grouped = frame['data2'].groupby(grouping)
grouped.apply(lambda x: x.describe()).unstack()

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
0,250.0,-0.057914,0.950495,-2.178644,-0.819267,-0.029644,0.538387,2.481195
1,250.0,-0.012960,1.045833,-2.473302,-0.735098,-0.035043,0.734594,2.724256
2,250.0,0.100901,0.983515,-2.755748,-0.551420,0.097412,0.749337,2.940230
3,250.0,-0.096310,1.049057,-2.747064,-0.723698,-0.172184,0.510550,2.473088


### Example: Filling Missing Values with Group-specific Values

In [76]:
s = Series(randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.137650
2         NaN
3    1.268427
4         NaN
5    0.200268
dtype: float64

In [77]:
s.fillna(s.mean())

0    0.443682
1   -0.137650
2    0.443682
3    1.268427
4    0.443682
5    0.200268
dtype: float64

In [78]:
chars = ['a','b','c','d','e','f','g','h']
group_key = ['red']*4 + ['blue']*4

data = Series(randn(8), index=chars)
data[['b','d','g']] = np.nan
data

a   -0.310348
b         NaN
c   -0.813753
d         NaN
e   -1.376191
f   -1.571477
g         NaN
h    0.908116
dtype: float64

In [84]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

a   -0.310348
b   -0.562050
c   -0.813753
d   -0.562050
e   -1.376191
f   -1.571477
g   -0.679851
h    0.908116
dtype: float64

In [85]:
fill_values = {'red': -0.5, 'blue': 0.5}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

a   -0.310348
b   -0.500000
c   -0.813753
d   -0.500000
e   -1.376191
f   -1.571477
g    0.500000
h    0.908116
dtype: float64

### Example: Random Sampling and Permutation

In [ ]:
suits = ['H', 'S', 'C', 'D']